## Lab 7-1: Tips

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
# For reproducibility
torch.manual_seed(1)

#### Training and Test Datasets


In [4]:
x_train = torch.FloatTensor([[1, 2, 1],
                             [1, 3, 2],
                             [1, 3, 4],
                             [1, 5, 5],
                             [1, 7, 5],
                             [1, 2, 5],
                             [1, 6, 6],
                             [1, 7, 7]
                            ])
y_train = torch.LongTensor([2, 2, 2, 1, 1, 1, 0, 0]) #정답

In [9]:
x_test = torch.FloatTensor([[2, 1, 1], 
                            [3, 1, 2], 
                            [3, 3, 4]])
y_test = torch.LongTensor([2, 2, 2])

print(y_test, y_test.shape) #정답
print(x_test, x_test.shape)

tensor([2, 2, 2]) torch.Size([3])
tensor([[2., 1., 1.],
        [3., 1., 2.],
        [3., 3., 4.]]) torch.Size([3, 3])


### Model

In [16]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3,3)  # x의 특성이 3개, ouput 개수는 3개 -->(3,3)
    def forward(self,x):
        return self.linear(x)

In [17]:
model = SoftmaxClassifierModel()

In [18]:
#optimizer 설정
optimizer = optim.SGD(model.parameters(), lr = 0.1)

In [42]:
def train(model, optimizer, x_train, y_train):
    nb_epochs =20
    
    for epoch in range(nb_epochs):
        
        #H(x)계산
        prediction = model(x_train)
        
        #cost 계산
        cost = F.cross_entropy(prediction, y_train)
        
        #cost로 h(x) 개선하기
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))
        
def test(model, optimizer, x_test, y_test):
    prediction = model(x_test)
    predicted_classes = prediction.max(1)[1]
    print('prediction의 값')
    print(prediction)
    print('\n prediction.max(1)의 값')
    print(prediction.max(1))
    print('\n prediction.max(1)[0]의 값')
    print(prediction.max(1)[0])
    print('\n prediction.max(1)[1]의 값')
    print(predicted_classes)
    
    correct_count = (predicted_classes == y_test).sum().item() #맞은 개수
    cost = F.cross_entropy(prediction, y_test) #cost 함수
    
    print('\n Accuracy: {}% Cost: {:.6f}'.format(
         correct_count / len(y_test) * 100, cost.item()
    ))

In [43]:
train(model, optimizer, x_train,y_train)

Epoch    0/20 Cost: 0.666434
Epoch    1/20 Cost: 0.665238
Epoch    2/20 Cost: 0.664053
Epoch    3/20 Cost: 0.662877
Epoch    4/20 Cost: 0.661712
Epoch    5/20 Cost: 0.660556
Epoch    6/20 Cost: 0.659409
Epoch    7/20 Cost: 0.658273
Epoch    8/20 Cost: 0.657145
Epoch    9/20 Cost: 0.656027
Epoch   10/20 Cost: 0.654917
Epoch   11/20 Cost: 0.653817
Epoch   12/20 Cost: 0.652726
Epoch   13/20 Cost: 0.651643
Epoch   14/20 Cost: 0.650568
Epoch   15/20 Cost: 0.649502
Epoch   16/20 Cost: 0.648444
Epoch   17/20 Cost: 0.647395
Epoch   18/20 Cost: 0.646353
Epoch   19/20 Cost: 0.645320


In [44]:
test(model, optimizer, x_test, y_test)

prediction의 값
tensor([[-2.3951, -0.5665,  3.3732],
        [-2.9070, -0.6166,  4.3394],
        [-2.1887, -0.4207,  2.9832]], grad_fn=<AddmmBackward>)

 prediction.max(1)의 값
torch.return_types.max(
values=tensor([3.3732, 4.3394, 2.9832], grad_fn=<MaxBackward0>),
indices=tensor([2, 2, 2]))

 prediction.max(1)[0]의 값
tensor([3.3732, 4.3394, 2.9832], grad_fn=<MaxBackward0>)

 prediction.max(1)[1]의 값
tensor([2, 2, 2])

 Accuracy: 100.0% Cost: 0.022744


### Learning Rate

Gradient Descent 에서의 $\alpha$ 값

optimizer = optim.SGD(model.parameters(), lr=0.1) 에서 lr=0.1 이다

learning rate이 너무 크면 diverge 하면서 cost 가 점점 늘어난다 (overshooting).

##### 반면에 learning rate이 너무 작으면 cost가 거의 줄어들지 않는다.

#### 보통 learning rate 은 0.1!!

### Data Preprocessing (데이터 전처리)


In [46]:
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

데이터를 zero-center하고 normalize하자.

여기서 $\sigma$ 는 standard deviation, $\mu$ 는 평균값 이다

In [48]:
mu = x_train.mean(dim=0)

In [49]:
sigma = x_train.std(dim=0)

In [52]:
#정규화하는 작업

norm_x_train = (x_train - mu) / sigma
print(norm_x_train)

tensor([[-1.0674, -0.3758, -0.8398],
        [ 0.7418,  0.2778,  0.5863],
        [ 0.3799,  0.5229,  0.3486],
        [ 1.0132,  1.0948,  1.1409],
        [-1.0674, -1.5197, -1.2360]])


In [53]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)

    def forward(self, x):
        return self.linear(x)

In [55]:
model = MultivariateLinearRegressionModel()
optimizer = optim.SGD(model.parameters(), lr=1e-1)

In [56]:
def train(model, optimizer, x_train, y_train):
    nb_epochs = 20
    for epoch in range(nb_epochs):

        # H(x) 계산
        prediction = model(x_train)

        # cost 계산
        cost = F.mse_loss(prediction, y_train)

        # cost로 H(x) 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

In [57]:
train(model, optimizer, norm_x_train, y_train)

Epoch    0/20 Cost: 29476.849609
Epoch    1/20 Cost: 18718.511719
Epoch    2/20 Cost: 11937.442383
Epoch    3/20 Cost: 7627.894531
Epoch    4/20 Cost: 4878.606445
Epoch    5/20 Cost: 3121.619873
Epoch    6/20 Cost: 1997.883789
Epoch    7/20 Cost: 1278.898926
Epoch    8/20 Cost: 818.800293
Epoch    9/20 Cost: 524.344238
Epoch   10/20 Cost: 335.887543
Epoch   11/20 Cost: 215.266602
Epoch   12/20 Cost: 138.059769
Epoch   13/20 Cost: 88.637863
Epoch   14/20 Cost: 56.998798
Epoch   15/20 Cost: 36.740959
Epoch   16/20 Cost: 23.767536
Epoch   17/20 Cost: 15.456335
Epoch   18/20 Cost: 10.129311
Epoch   19/20 Cost: 6.712480


### Overfitting

너무 학습 데이터에 한해 잘 학습해 테스트 데이터에 좋은 성능을 내지 못할 수도 있다.



이것을 방지하는 방법은 크게 세 가지:

- 더 많은 학습 데이터
- 더 적은 양의 feature
- Regularization (너무 큰 숫자를 가중치에 두지 말자)

In [59]:
def train_with_regularization(model, optimizer, x_train, y_train):
    nb_epochs = 20
    for epoch in range(nb_epochs):

        # H(x) 계산
        prediction = model(x_train)

        # cost 계산
        cost = F.mse_loss(prediction, y_train)
        
        # l2 norm 계산
        l2_reg = 0
        for param in model.parameters():
            l2_reg += torch.norm(param)
            
        cost += l2_reg

        # cost로 H(x) 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch+1, nb_epochs, cost.item()
        ))

In [61]:
train_with_regularization(model, optimizer, norm_x_train, y_train)

Epoch    1/20 Cost: 187.072037
Epoch    2/20 Cost: 186.205078
Epoch    3/20 Cost: 185.671631
Epoch    4/20 Cost: 185.332962
Epoch    5/20 Cost: 185.113693
Epoch    6/20 Cost: 184.969498
Epoch    7/20 Cost: 184.873093
Epoch    8/20 Cost: 184.807373
Epoch    9/20 Cost: 184.761459
Epoch   10/20 Cost: 184.728424
Epoch   11/20 Cost: 184.703812
Epoch   12/20 Cost: 184.684753
Epoch   13/20 Cost: 184.669449
Epoch   14/20 Cost: 184.656708
Epoch   15/20 Cost: 184.645737
Epoch   16/20 Cost: 184.636063
Epoch   17/20 Cost: 184.627380
Epoch   18/20 Cost: 184.619415
Epoch   19/20 Cost: 184.612076
Epoch   20/20 Cost: 184.605240
